In [ ]:
import logging
from conf import LisaLogging
LisaLogging.setup()

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%pylab inline

import json
import os

# Support to access the remote target
import devlib
from env import TestEnv

# Import support for Android devices
from android import Screen, Workload

# Support for trace events analysis
from trace import Trace

# Suport for FTrace events parsing and visualization
import trappy
import pandas as pd

from wlgen import RTA, Periodic, Ramp
from time import sleep

from IPython.display import display

In [ ]:
linux_hikey = {
    "platform" : "linux",
    "board" : "hikey960",
    "host": "10.42.0.1",
    "username" : "root",
    "password" : "root",
    "modules" : ["sched", "cgroups", "hotplug"],
    "tools" : ["taskset", "rt-app"],
    "rtapp-calib" :  {
        "0": 302, "1": 302, "2": 302, "3": 302, "4": 136, "5": 136, "6": 136, "7": 136
     }
}

In [ ]:
linux_juno = {
    "platform" : "linux",
    "board" : "juno",
    "host": "192.168.0.1",
    "username" : "root",
    "password" : "root",
    "modules" : ["sched", "cgroups", "hotplug"],
    "tools" : ["taskset", "rt-app"]
}

In [ ]:
android_hikey = {
    "platform" : "android",
    "board" : "hikey960",
    "modules" : ["sched", "hotplug"],
    "tools" : ["taskset", "rt-app"]
}

In [ ]:
te = TestEnv(linux_hikey)
target = te.target

### Generics

In [ ]:
from libs.utils.kernel_tests.scheduler.eas_behaviour import *

In [ ]:
tests = [OneSmallTask, ThreeSmallTasks, TwoBigTasks, TwoBigThreeSmall, RampUp, RampDown, EnergyModelWakeMigration]

In [ ]:
res = {}

for test in tests:
    bundle = test.from_target(te)
    res[test.__name__] = bundle

In [ ]:
for test_name, bundle in res.iteritems():
    print test_name
    print "---"
    print "test_slack: {}".format("PASSED" if bundle.test_slack() else "FAILED")
    print "test_task_placement: {}".format("PASSED" if bundle.test_task_placement() else "FAILED")
    print bundle.test_task_placement()
    print

### CPU sanity check

In [ ]:
from libs.utils.kernel_tests.misc.capacity_sanity import CapacitySanityCheck

In [ ]:
bundle = CapacitySanityCheck.from_target(te)

In [ ]:
if bundle.test_capacity_sanity():
    print "PASSED"
else:
    print "FAILED"

print bundle.test_capacity_sanity()

### Hotplug

In [ ]:
from libs.utils.kernel_tests.hotplug.torture import HotplugTorture

In [ ]:
bundle = HotplugTorture.from_target(te)

In [ ]:
bundle.test_cpus_alive().passed

In [ ]:
bundle.test_cpus_alive().metrics

In [ ]:
bundle.test_target_alive().passed